# ಸೆಷನ್ 4 – SLM ಮತ್ತು LLM ಹೋಲಿಕೆ

ಫೌಂಡ್ರಿ ಲೋಕಲ್ ಮೂಲಕ ಚಾಲನೆ ಮಾಡುವ ಸಣ್ಣ ಭಾಷಾ ಮಾದರಿ ಮತ್ತು ದೊಡ್ಡ ಮಾದರಿಯ ನಡುವೆ ವಿಳಂಬ ಮತ್ತು ಮಾದರಿ ಪ್ರತಿಕ್ರಿಯೆಯ ಗುಣಮಟ್ಟವನ್ನು ಹೋಲಿಸಿ.


## ⚡ ತ್ವರಿತ ಪ್ರಾರಂಭ

**ಮೆಮೊರಿ-ಆಪ್ಟಿಮೈಜ್ಡ್ ಸೆಟಪ್ (ನವೀಕರಿಸಲಾಗಿದೆ):**
1. ಮಾದರಿಗಳು ಸ್ವಯಂಚಾಲಿತವಾಗಿ CPU ಬದಲಾವಣೆಗಳನ್ನು ಆಯ್ಕೆಮಾಡುತ್ತವೆ (ಯಾವುದೇ ಹಾರ್ಡ್‌ವೇರ್‌ನಲ್ಲಿ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ)
2. 7B ಬದಲು `qwen2.5-3b` ಬಳಸುತ್ತದೆ (~4GB RAM ಉಳಿಸುತ್ತದೆ)
3. ಪೋರ್ಟ್ ಸ್ವಯಂ-ಗುರುತಿಸುವಿಕೆ (ಮ್ಯಾನುಯಲ್ ಸಂರಚನೆ ಅಗತ್ಯವಿಲ್ಲ)
4. ಒಟ್ಟು RAM ಅಗತ್ಯ: ~8GB ಶಿಫಾರಸು ಮಾಡಲಾಗಿದೆ (ಮಾದರಿಗಳು + OS)

**ಟರ್ಮಿನಲ್ ಸೆಟಪ್ (30 ಸೆಕೆಂಡುಗಳು):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

ನಂತರ ಈ ನೋಟ್ಬುಕ್ ಅನ್ನು ಚಾಲನೆ ಮಾಡಿ! 🚀


### ವಿವರಣೆ: ಅವಲಂಬನೆ ಸ್ಥಾಪನೆ
ಟೈಮಿಂಗ್ ಮತ್ತು ಚಾಟ್ ವಿನಂತಿಗಳಿಗಾಗಿ ಅಗತ್ಯವಿರುವ ಕನಿಷ್ಠ ಪ್ಯಾಕೇಜುಗಳು (`foundry-local-sdk`, `openai`, `numpy`) ಅನ್ನು ಸ್ಥಾಪಿಸುತ್ತದೆ. ಪುನಃ ನಿರ್ವಹಿಸಲು ಸುರಕ್ಷಿತವಾಗಿದೆ.


# ದೃಶ್ಯಾವಳಿ
ಪ್ರತಿನಿಧಿ ಸಣ್ಣ ಭಾಷಾ ಮಾದರಿಯನ್ನು (SLM) ದೊಡ್ಡ ಮಾದರಿಯೊಂದಿಗಿನ ಒಬ್ಬೇ ಪ್ರಾಂಪ್ಟ್‌ನಲ್ಲಿ ಹೋಲಿಸಿ ವ್ಯಾಪ್ತಿಗಳನ್ನು ವಿವರಿಸಲು:
- **ವಿಲಂಬದ ವ್ಯತ್ಯಾಸ** (ಗಡಿಯಾರ ಸೆಕೆಂಡುಗಳು)
- **ಟೋಕನ್ ಬಳಕೆ** (ಲಭ್ಯವಿದ್ದರೆ) throughput ಗೆ ಪ್ರಾಕ್ಸಿ ಆಗಿ
- **ತ್ವರಿತ ದೃಷ್ಟಿಗೆ ಮಾದರಿ ಗುಣಮಟ್ಟದ ಔಟ್‌ಪುಟ್**
- **ವೇಗದ ಲೆಕ್ಕಾಚಾರ** ಕಾರ್ಯಕ್ಷಮತೆ ಲಾಭಗಳನ್ನು ಪ್ರಮಾಣೀಕರಿಸಲು

**ಪರಿಸರ ಚರಗಳು:**
- `SLM_ALIAS` - ಸಣ್ಣ ಭಾಷಾ ಮಾದರಿ (ಡೀಫಾಲ್ಟ್: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - ದೊಡ್ಡ ಭಾಷಾ ಮಾದರಿ (ಡೀಫಾಲ್ಟ್: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - ಹೋಲಿಕೆಗಾಗಿ ಪರೀಕ್ಷಾ ಪ್ರಾಂಪ್ಟ್
- `COMPARE_RETRIES` - ಮರುಪ್ರಯತ್ನಗಳ ಸಂಖ್ಯೆ (ಡೀಫಾಲ್ಟ್: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - ಸೇವಾ ಎಂಡ್‌ಪಾಯಿಂಟ್ ಅನ್ನು ಮೀರಿಸುವುದು (ಸೆಟ್ ಮಾಡದಿದ್ದರೆ ಸ್ವಯಂಚಾಲಿತ ಪತ್ತೆ)

**ಇದು ಹೇಗೆ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ (ಅಧಿಕೃತ SDK ಮಾದರಿ):**
1. **FoundryLocalManager** Foundry Local ಸೇವೆಯನ್ನು ಪ್ರಾರಂಭಿಸಿ ನಿರ್ವಹಿಸುತ್ತದೆ
2. ಸೇವೆ ಚಾಲನೆಯಲ್ಲದಿದ್ದರೆ ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಪ್ರಾರಂಭವಾಗುತ್ತದೆ (ಮ್ಯಾನುಯಲ್ ಸೆಟ್‌ಅಪ್ ಅಗತ್ಯವಿಲ್ಲ)
3. ಮಾದರಿಗಳನ್ನು ಅಲಿಯಾಸ್‌ಗಳಿಂದ ನಿಖರ ID ಗಳಿಗೆ ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಪರಿಹರಿಸಲಾಗುತ್ತದೆ
4. ಹಾರ್ಡ್‌ವೇರ್-ಆಪ್ಟಿಮೈಸ್ ಮಾಡಿದ ರೂಪಾಂತರಗಳನ್ನು ಆಯ್ಕೆ ಮಾಡಲಾಗುತ್ತದೆ (CUDA, NPU, ಅಥವಾ CPU)
5. OpenAI-ಸಮ್ಮತ ಕ್ಲೈಂಟ್ ಚಾಟ್ ಪೂರ್ಣಗೊಳಿಸುವಿಕೆಗಳನ್ನು ನಿರ್ವಹಿಸುತ್ತದೆ
6. ಮೆಟ್ರಿಕ್ಸ್ ಗಳನ್ನು ಸೆರೆಹಿಡಿಯಲಾಗುತ್ತದೆ: ವಿಲಂಬ, ಟೋಕನ್ಗಳು, ಔಟ್‌ಪುಟ್ ಗುಣಮಟ್ಟ
7. ಫಲಿತಾಂಶಗಳನ್ನು ಹೋಲಿಸಿ ವೇಗದ ಅನುಪಾತ ಲೆಕ್ಕಿಸಲಾಗುತ್ತದೆ

ಈ ಸೂಕ್ಷ್ಮ ಹೋಲಿಕೆ ನಿಮ್ಮ ಬಳಕೆಗೆ ದೊಡ್ಡ ಮಾದರಿಯ ಕಡೆ ಮಾರ್ಗದರ್ಶನ ಮಾಡಬೇಕಾಗಿರುವಾಗ ನಿರ್ಧರಿಸಲು ಸಹಾಯ ಮಾಡುತ್ತದೆ.

**SDK ಉಲ್ಲೇಖ:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- ವರ್ಕ್‌ಶಾಪ್ ಉಪಕರಣಗಳು: ../samples/workshop_utils.py ನಿಂದ ಅಧಿಕೃತ ಮಾದರಿಯನ್ನು ಬಳಸುತ್ತದೆ

**ಪ್ರಮುಖ ಲಾಭಗಳು:**
- ✅ ಸ್ವಯಂಚಾಲಿತ ಸೇವೆ ಪತ್ತೆ ಮತ್ತು ಪ್ರಾರಂಭ
- ✅ ಸೇವೆ ಚಾಲನೆಯಲ್ಲದಿದ್ದರೆ ಸ್ವಯಂಚಾಲಿತ ಪ್ರಾರಂಭ
- ✅ ನಿರ್ಮಿತ ಮಾದರಿ ಪರಿಹಾರ ಮತ್ತು ಕ್ಯಾಶಿಂಗ್
- ✅ ಹಾರ್ಡ್‌ವೇರ್ ಆಪ್ಟಿಮೈಜೆಷನ್ (CUDA/NPU/CPU)
- ✅ OpenAI SDK ಹೊಂದಾಣಿಕೆ
- ✅ ಮರುಪ್ರಯತ್ನಗಳೊಂದಿಗೆ ದೃಢವಾದ ದೋಷ ನಿರ್ವಹಣೆ
- ✅ ಸ್ಥಳೀಯ ನಿರ್ಣಯ (ಮೇಘ API ಅಗತ್ಯವಿಲ್ಲ)


## 🚨 ಪೂರ್ವಾಪೇಕ್ಷೆಗಳು: ಫೌಂಡ್ರಿ ಲೋಕಲ್ ಚಾಲನೆಯಲ್ಲಿರಬೇಕು!

**ಈ ನೋಟ್ಬುಕ್ ಅನ್ನು ಚಾಲನೆ ಮಾಡುವ ಮೊದಲು**, ಫೌಂಡ್ರಿ ಲೋಕಲ್ ಸೇವೆ ಸ್ಥಾಪಿತವಾಗಿದೆ ಎಂದು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಿ:

### ತ್ವರಿತ ಪ್ರಾರಂಭ ಆಜ್ಞೆಗಳು (ಟರ್ಮಿನಲ್‌ನಲ್ಲಿ ಚಾಲನೆ ಮಾಡಿ):

```bash
# 1. ಫೌಂಡ್ರಿ ಲೋಕಲ್ ಸೇವೆಯನ್ನು ಪ್ರಾರಂಭಿಸಿ
foundry service start

# 2. ಈ ಹೋಲಿಕೆಯಲ್ಲಿ ಬಳಸುವ ಡೀಫಾಲ್ಟ್ ಮಾದರಿಗಳನ್ನು ಲೋಡ್ ಮಾಡಿ (CPU-ಆಪ್ಟಿಮೈಸ್ ಮಾಡಲಾಗಿದೆ)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. ಮಾದರಿಗಳು ಲೋಡ್ ಆಗಿರುವುದನ್ನು ಪರಿಶೀಲಿಸಿ
foundry model ls

# 4. ಸೇವೆಯ ಆರೋಗ್ಯವನ್ನು ಪರಿಶೀಲಿಸಿ
foundry service status
```

### ಪರ್ಯಾಯ ಮಾದರಿಗಳು (ಡೀಫಾಲ್ಟ್‌ಗಳು ಲಭ್ಯವಿಲ್ಲದಿದ್ದರೆ):

```bash
# ಇನ್ನೂ ಚಿಕ್ಕ ಪರ್ಯಾಯಗಳು (ಸ್ಮೃತಿ ಬಹಳ ಸೀಮಿತವಾಗಿದ್ದರೆ)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# ಅಥವಾ ಈ ನೋಟ್ಬುಕ್‌ನಲ್ಲಿ ಪರಿಸರ ಚರಗಳನ್ನು ನವೀಕರಿಸಿ:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # ಅಥವಾ ಕನಿಷ್ಠ ಸ್ಮೃತಿಗಾಗಿ qwen2.5-0.5b
```

⚠️ **ನೀವು ಈ ಹಂತಗಳನ್ನು ಬಿಟ್ಟುಬಿಟ್ಟರೆ**, ಕೆಳಗಿನ ನೋಟ್ಬುಕ್ ಸೆಲ್‌ಗಳನ್ನು ಚಾಲನೆ ಮಾಡುವಾಗ `APIConnectionError` ಕಾಣಿಸಿಕೊಳ್ಳುತ್ತದೆ.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### ವಿವರಣೆ: ಕೋರ್ ಆಮದುಗಳು
ಮಾದರಿ ಮಾಹಿತಿಯನ್ನು ಪಡೆಯಲು ಮತ್ತು ಚಾಟ್ ಪೂರ್ಣಗೊಳಿಸುವಿಕೆಗಳನ್ನು ನಿರ್ವಹಿಸಲು ಬಳಸುವ ಟೈಮಿಂಗ್ ಉಪಕರಣಗಳು ಮತ್ತು Foundry Local / OpenAI ಕ್ಲೈಂಟ್‌ಗಳನ್ನು ಒಳಗೊಂಡಿದೆ.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### ವಿವರಣೆ: ಉಪನಾಮಗಳು ಮತ್ತು ಪ್ರಾಂಪ್ಟ್ ಸೆಟ್‌ಅಪ್  
ಸಣ್ಣ ಮತ್ತು ದೊಡ್ಡ ಮಾದರಿಗಾಗಿ ಪರಿಸರ-ಕಾನ್ಫಿಗರ್ ಮಾಡಬಹುದಾದ ಉಪನಾಮಗಳನ್ನು ಮತ್ತು ಹೋಲಿಕೆ ಪ್ರಾಂಪ್ಟ್ ಅನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುತ್ತದೆ. ವಿಭಿನ್ನ ಮಾದರಿ ಕುಟುಂಬಗಳು ಅಥವಾ ಕಾರ್ಯಗಳೊಂದಿಗೆ ಪ್ರಯೋಗ ಮಾಡಲು ಪರಿಸರ ಚರಗಳನ್ನು ಹೊಂದಿಸಿ.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 ಮೆಮೊರಿ-ಆಪ್ಟಿಮೈಸ್ ಮಾಡಿದ ಸಂರಚನೆ

**ಈ ನೋಟ್ಬುಕ್ ಡೀಫಾಲ್ಟ್ ಆಗಿ ಮೆಮೊರಿ-ಕಾರ್ಯಕ್ಷಮ ಮಾದರಿಗಳನ್ನು ಬಳಸುತ್ತದೆ:**
- `phi-4-mini` → ~4GB RAM (Foundry Local ಸ್ವಯಂಚಾಲಿತವಾಗಿ CPU ಆವೃತ್ತಿಯನ್ನು ಆಯ್ಕೆಮಾಡುತ್ತದೆ)
- `qwen2.5-3b` → ~3GB RAM (7B ಬದಲು, ಅದಕ್ಕೆ ~7GB+ ಬೇಕಾಗುತ್ತದೆ)

**ಪೋರ್ಟ್ ಸ್ವಯಂ-ಗುರುತಿಸುವಿಕೆ:**
- Foundry Local ವಿಭಿನ್ನ ಪೋರ್ಟ್‌ಗಳನ್ನು ಬಳಸಬಹುದು (ಸಾಮಾನ್ಯವಾಗಿ 55769 ಅಥವಾ 59959)
- ಕೆಳಗಿನ ಡಯಾಗ್ನೋಸ್ಟಿಕ್ ಸೆಲ್ ಸರಿಯಾದ ಪೋರ್ಟ್ ಅನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಗುರುತಿಸುತ್ತದೆ
- ಕೈಯಿಂದ ಸಂರಚನೆ ಅಗತ್ಯವಿಲ್ಲ!

**ನಿಮ್ಮ ಬಳಿ ಸೀಮಿತ RAM (<8GB) ಇದ್ದರೆ, ಇನ್ನೂ ಚಿಕ್ಕ ಮಾದರಿಗಳನ್ನು ಬಳಸಿ:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### ವಿವರಣೆ: ಕಾರ್ಯಾಚರಣೆ ಸಹಾಯಕರು (Foundry SDK ಮಾದರಿ)
ಕಾರ್ಯಾಗಾರ ಮಾದರಿಗಳಲ್ಲಿ ದಾಖಲಾಗಿರುವ ಅಧಿಕೃತ Foundry Local SDK ಮಾದರಿಯನ್ನು ಬಳಸುತ್ತದೆ:

**ವಿಧಾನ:**
- **FoundryLocalManager** - Foundry Local ಸೇವೆಯನ್ನು ಪ್ರಾರಂಭಿಸಿ ನಿರ್ವಹಿಸುತ್ತದೆ
- **ಸ್ವಯಂಚಾಲಿತ ಪತ್ತೆ** - ಅಂತಿಮ ಬಿಂದುವನ್ನು ಸ್ವಯಂಚಾಲಿತವಾಗಿ ಕಂಡುಹಿಡಿದು ಸೇವೆಯ ಜೀವನಚಕ್ರವನ್ನು ನಿರ್ವಹಿಸುತ್ತದೆ
- **ಮಾದರಿ ಪರಿಹಾರ** - ಪೂರ್ಣ ಮಾದರಿ ID ಗಳಿಗೆ ಉಪನಾಮಗಳನ್ನು ಪರಿಹರಿಸುತ್ತದೆ (ಉದಾ: phi-4-mini → phi-4-mini-instruct-cpu)
- **ಹಾರ್ಡ್‌ವೇರ್ ಆಪ್ಟಿಮೈಜೆಷನ್** - ಲಭ್ಯವಿರುವ ಹಾರ್ಡ್‌ವೇರ್ (CUDA, NPU, ಅಥವಾ CPU) ಗೆ ಉತ್ತಮ ಪರ್ಯಾಯವನ್ನು ಆಯ್ಕೆಮಾಡುತ್ತದೆ
- **OpenAI ಕ್ಲೈಂಟ್** - OpenAI-ಸಮ್ಮತ API ಪ್ರವೇಶಕ್ಕಾಗಿ ಮ್ಯಾನೇಜರ್‌ನ ಅಂತಿಮ ಬಿಂದುವಿನಿಂದ ಸಂರಚಿತವಾಗಿದೆ

**ಸ್ಥೈರ್ಯ ವೈಶಿಷ್ಟ್ಯಗಳು:**
- ಘಾತಾಂಕ ಬ್ಯಾಕ್ಓಫ್ ಮರುಪ್ರಯತ್ನ ತರ್ಕ (ಪರಿಸರದಿಂದ ಸಂರಚನೀಯ)
- ಸೇವೆ ಚಾಲನೆಯಲ್ಲದಿದ್ದರೆ ಸ್ವಯಂಚಾಲಿತ ಸೇವಾ ಪ್ರಾರಂಭ
- ಪ್ರಾರಂಭದ ನಂತರ ಸಂಪರ್ಕ ಪರಿಶೀಲನೆ
- ವಿವರವಾದ ದೋಷ ವರದಿಯೊಂದಿಗೆ ಸೌಮ್ಯ ದೋಷ ನಿರ್ವಹಣೆ
- ಪುನರಾವರ್ತಿತ ಪ್ರಾರಂಭವನ್ನು ತಪ್ಪಿಸಲು ಮಾದರಿ ಕ್ಯಾಶಿಂಗ್

**ಫಲಿತಾಂಶ ರಚನೆ:**
- ವಿಳಂಬ ಅಳೆಯುವಿಕೆ (ಗಡಿಯಾರ ಸಮಯ)
- ಟೋಕನ್ ಬಳಕೆ ಟ್ರ್ಯಾಕಿಂಗ್ (ಲಭ್ಯವಿದ್ದರೆ)
- ಮಾದರಿ ಔಟ್‌ಪುಟ್ (ಓದಲು ಸುಲಭವಾಗಿಸಲು ಕಡಿತಗೊಳಿಸಲಾಗಿದೆ)
- ವಿಫಲವಾದ ವಿನಂತಿಗಳ ದೋಷ ವಿವರಗಳು

ಈ ಮಾದರಿ ಅಧಿಕೃತ SDK ಮಾದರಿಯನ್ನು ಅನುಸರಿಸುವ workshop_utils ಮೋಡ್ಯೂಲ್ ಅನ್ನು ಬಳಸುತ್ತದೆ.

**SDK ಉಲ್ಲೇಖ:**
- ಮುಖ್ಯ ರೆಪೊ: https://github.com/microsoft/Foundry-Local
- ಪೈಥಾನ್ SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- ಕಾರ್ಯಾಗಾರ ಉಪಕರಣಗಳು: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### ವಿವರಣೆ: ಪೂರ್ವ-ಪ್ರಯಾಣ ಸ್ವ-ಪರೀಕ್ಷೆ
ಎರಡೂ ಮಾದರಿಗಳಿಗಾಗಿ FoundryLocalManager ಬಳಸಿ ಲಘು ತೂಕದ ಸಂಪರ್ಕ ಪರಿಶೀಲನೆಯನ್ನು ನಡೆಸುತ್ತದೆ. ಇದು ಪರಿಶೀಲಿಸುತ್ತದೆ:
- ಸೇವೆ ಲಭ್ಯವಿದೆ
- ಮಾದರಿಗಳನ್ನು ಪ್ರಾರಂಭಿಸಬಹುದು
- ಉಪನಾಮಗಳು ನಿಜವಾದ ಮಾದರಿ ID ಗಳಿಗೆ ಪರಿಹರಿಸಲಾಗುತ್ತದೆ
- ಹೋಲಿಕೆಯನ್ನು ನಡೆಸುವ ಮೊದಲು ಸಂಪರ್ಕ ಸ್ಥಿರವಾಗಿದೆ

setup() ಫಂಕ್ಷನ್ ಅಧಿಕೃತ SDK ಮಾದರಿಯನ್ನು workshop_utils ನಿಂದ ಬಳಸುತ್ತದೆ.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ ತ್ವರಿತ ಪರಿಹಾರ: ನೋಟ್ಬುಕ್‌ನಿಂದ ಫೌಂಡ್ರಿ ಲೋಕಲ್ ಪ್ರಾರಂಭಿಸಿ (ಐಚ್ಛಿಕ)

ಮೇಲಿನ ಡಯಾಗ್ನೋಸ್ಟಿಕ್ ಸೇವೆ ಚಾಲನೆಯಲ್ಲದಿದ್ದರೆ, ನೀವು ಇದರಿಂದ ಪ್ರಾರಂಭಿಸಲು ಪ್ರಯತ್ನಿಸಬಹುದು:

**ಗಮನಿಸಿ:** ಇದು ವಿಂಡೋಸ್‌ನಲ್ಲಿ ಉತ್ತಮವಾಗಿ ಕಾರ್ಯನಿರ್ವಹಿಸುತ್ತದೆ. ಇತರ ವೇದಿಕೆಗಳಲ್ಲಿ, ಟರ್ಮಿನಲ್ ಆಜ್ಞೆಗಳನ್ನು ಬಳಸಿ.


### ⚠️ ಸಂಪರ್ಕ ದೋಷಗಳನ್ನು ಪರಿಹರಿಸುವುದು

ನೀವು `APIConnectionError` ಅನ್ನು ನೋಡುತ್ತಿದ್ದರೆ, Foundry Local ಸೇವೆ ಚಾಲನೆಯಲ್ಲಿರಲಿಲ್ಲ ಅಥವಾ ಮಾದರಿಗಳು ಲೋಡ್ ಆಗಿಲ್ಲ. ಈ ಹಂತಗಳನ್ನು ಪ್ರಯತ್ನಿಸಿ:

**1. ಸೇವೆಯ ಸ್ಥಿತಿಯನ್ನು ಪರಿಶೀಲಿಸಿ:**
```bash
# ಟರ್ಮಿನಲ್‌ನಲ್ಲಿ (ನೋಟ್‌ಬುಕ್‌ನಲ್ಲಿ ಅಲ್ಲ):
foundry service status
```

**2. ಸೇವೆಯನ್ನು ಪ್ರಾರಂಭಿಸಿ (ಚಾಲನೆಯಲ್ಲದಿದ್ದರೆ):**
```bash
foundry service start
```

**3. ಅಗತ್ಯವಿರುವ ಮಾದರಿಗಳನ್ನು ಲೋಡ್ ಮಾಡಿ:**
```bash
# ಹೋಲಿಕೆಗಾಗಿ ಅಗತ್ಯವಿರುವ ಮಾದರಿಗಳನ್ನು ಲೋಡ್ ಮಾಡಿ
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# ಅಥವಾ ಪರ್ಯಾಯ ಮಾದರಿಗಳನ್ನು ಬಳಸಿ:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. ಮಾದರಿಗಳು ಲಭ್ಯವಿರುವುದನ್ನು ಪರಿಶೀಲಿಸಿ:**
```bash
foundry model ls
```

**ಸಾಮಾನ್ಯ ಸಮಸ್ಯೆಗಳು:**
- ❌ ಸೇವೆ ಚಾಲನೆಯಲ್ಲಿಲ್ಲ → `foundry service start` ಅನ್ನು ಚಾಲನೆ ಮಾಡಿ
- ❌ ಮಾದರಿಗಳು ಲೋಡ್ ಆಗಿಲ್ಲ → `foundry model run <model-name>` ಅನ್ನು ಚಾಲನೆ ಮಾಡಿ
- ❌ ಪೋರ್ಟ್ ಸಂಘರ್ಷಗಳು → ಇನ್ನೊಂದು ಸೇವೆ ಪೋರ್ಟ್ ಬಳಸುತ್ತಿದ್ದರೆ ಪರಿಶೀಲಿಸಿ
- ❌ ಫೈರ್‌ವಾಲ್ ತಡೆಹಿಡಿಯುತ್ತಿದೆ → ಸ್ಥಳೀಯ ಸಂಪರ್ಕಗಳಿಗೆ ಅನುಮತಿ ನೀಡಲಾಗಿದೆ ಎಂದು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಿ

**ತ್ವರಿತ ಪರಿಹಾರ:** ಪೂರ್ವ-ಪರೀಕ್ಷೆ ಮೊದಲು ಕೆಳಗಿನ ಡಯಾಗ್ನೋಸ್ಟಿಕ್ ಸೆಲ್ ಅನ್ನು ಚಾಲನೆ ಮಾಡಿ.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ ಪೂರ್ವ-ಪ್ರಯಾಣ ಪರಿಶೀಲನೆ: ಮಾದರಿ ಲಭ್ಯತೆ

ಈ ಸೆಲ್ ಹೋಲಿಕೆಯನ್ನು ನಡೆಸುವ ಮೊದಲು ಸಂರಚಿತ ಎಂಡ್‌ಪಾಯಿಂಟ್‌ನಲ್ಲಿ ಎರಡೂ ಮಾದರಿಗಳನ್ನು ತಲುಪಬಹುದೇ ಎಂದು ಪರಿಶೀಲಿಸುತ್ತದೆ.


### ವಿವರಣೆ: ರನ್ ಹೋಲಿಕೆ ಮತ್ತು ಫಲಿತಾಂಶಗಳನ್ನು ಸಂಗ್ರಹಿಸಿ
ಅಧಿಕೃತ Foundry SDK ಮಾದರಿಯನ್ನು ಬಳಸಿ ಎರಡೂ ಉಪನಾಮಗಳ ಮೇಲೆ ಪುನರಾವರ್ತನೆ ಮಾಡುತ್ತದೆ:
1. ಪ್ರತಿ ಮಾದರಿಯನ್ನು setup() ಮೂಲಕ ಪ್ರಾರಂಭಿಸಿ (FoundryLocalManager ಅನ್ನು ಬಳಸುತ್ತದೆ)
2. OpenAI-ಸಮ್ಮತ API ಮೂಲಕ ನಿರ್ಣಯವನ್ನು ನಡೆಸಿ
3. ವಿಳಂಬ, ಟೋಕನ್ಗಳು ಮತ್ತು ಮಾದರಿ ಔಟ್‌ಪುಟ್ ಅನ್ನು ಹಿಡಿಯಿರಿ
4. ಹೋಲಿಕೆ ವಿಶ್ಲೇಷಣೆಯೊಂದಿಗೆ JSON ಸಾರಾಂಶವನ್ನು ಉತ್ಪಾದಿಸಿ

ಇದು session04/model_compare.py ನಲ್ಲಿ ವರ್ಕ್‌ಶಾಪ್ ಮಾದರಿಗಳಂತೆ ಅದೇ ಮಾದರಿಯನ್ನು ಅನುಸರಿಸುತ್ತದೆ.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### ಫಲಿತಾಂಶಗಳನ್ನು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವುದು

**ಪ್ರಮುಖ ಅಂಶಗಳು:**
- **ವಿಲಂಬ**: ಕಡಿಮೆ ಇದ್ದರೆ ಉತ್ತಮ - ವೇಗವಾದ ಪ್ರತಿಕ್ರಿಯೆ ಸಮಯವನ್ನು ಸೂಚಿಸುತ್ತದೆ
- **ಟೋಕನ್ಗಳು**: ಹೆಚ್ಚಿನ ಥ್ರೂಪುಟ್ = ಹೆಚ್ಚು ಟೋಕನ್ಗಳನ್ನು ಪ್ರಕ್ರಿಯೆ ಮಾಡಲಾಗಿದೆ
- **ರೌಟ್**: ಯಾವ API ಎಂಡ್‌ಪಾಯಿಂಟ್ ಬಳಸಲಾಗಿದೆ ಎಂದು ದೃಢಪಡಿಸುತ್ತದೆ

**SLM ಮತ್ತು LLM ಅನ್ನು ಯಾವಾಗ ಬಳಸಬೇಕು:**
- **SLM (ಸಣ್ಣ ಭಾಷಾ ಮಾದರಿ)**: ವೇಗವಾದ ಪ್ರತಿಕ್ರಿಯೆಗಳು, ಕಡಿಮೆ ಸಂಪನ್ಮೂಲ ಬಳಕೆ, ಸರಳ ಕಾರ್ಯಗಳಿಗೆ ಉತ್ತಮ
- **LLM (ದೊಡ್ಡ ಭಾಷಾ ಮಾದರಿ)**: ಹೆಚ್ಚಿನ ಗುಣಮಟ್ಟ, ಉತ್ತಮ ತರ್ಕ, ಗುಣಮಟ್ಟ ಮುಖ್ಯವಾಗಿರುವಾಗ ಬಳಸಬೇಕು

**ಮುಂದಿನ ಹಂತಗಳು:**
1. ಜಟಿಲತೆ ಹೋಲಿಕೆಗೆ ಪ್ರಾಂಪ್ಟ್‌ಗಳ ವಿಭಿನ್ನ ಪ್ರಯೋಗ ಮಾಡಿ
2. ಇತರ ಮಾದರಿ ಜೋಡಿಗಳೊಂದಿಗೆ ಪ್ರಯೋಗ ಮಾಡಿ
3. ಕಾರ್ಯ ಜಟಿಲತೆಯ ಆಧಾರದ ಮೇಲೆ ಬುದ್ಧಿವಂತಿಕೆಯಿಂದ ರೌಟ್ ಮಾಡಲು ವರ್ಕ್‌ಶಾಪ್ ರೌಟರ್ ಮಾದರಿಗಳನ್ನು (ಸೆಷನ್ 06) ಬಳಸಿ


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸತ್ಯತೆಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
